In [2]:
!pip install datasets flax jax -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached datasets-2.4.0-py3-none-any.whl (365 kB)
  Using cached flax-0.6.0-py3-none-any.whl (180 kB)
  Using cached multiprocess-0.70.13-py37-none-any.whl (115 kB)
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached xxhash-3.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (212 kB)
  Using cached huggingface_hub-0.9.1-py3-none-any.whl (120 kB)
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Using cached optax-0.1.3-py3-none-any.whl (145 kB)
  Using cached rich-11.2.0-py3-none-any.whl (217 kB)
  Using cached colorama-0.4.5-py2.py3-none-any.whl (16 kB)
  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)
     |████████████████████████████████| 76 kB 2.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
import jax
print(jax.default_backend())

gpu


In [4]:
print(jax.devices())

[GpuDevice(id=0, process_index=0)]


In [5]:
import numpy as np
from datasets import load_dataset
import jax.numpy as jnp
from flax.training import train_state
import optax
from tqdm.auto import tqdm

In [ ]:
# weight_after_train <- model, init_weight, data
# predict_label <- model(weight_after_train, predict_to_data)

In [6]:
dataset = load_dataset("mnist")

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset mnist downloaded and prepared to /root/.cache/huggingface/datasets/mnist/mnist/1.0.0/fda16c03c4ecfb13f165ba7e29cf38129ce035011519968cdaf74894ce91c9d4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
dataset['train']

Dataset({
    features: ['image', 'label'],
    num_rows: 60000
})

In [8]:
dataset['test']

Dataset({
    features: ['image', 'label'],
    num_rows: 10000
})

In [83]:
dataset['train'][59999]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x7F29221BAA50>,
 'label': 8}

In [84]:
dataset['train'][59999]['image']

In [10]:
len(dataset['train'])

60000

In [11]:
def flow_data(dataset, batch_size=32):
    total = len(dataset)
    total_batch = total // batch_size
    for i in range(total_batch):
        batch = dataset[i * batch_size : (i + 1) * batch_size]
        images = np.expand_dims(np.stack([np.array(x) for x in batch['image']]), -1).astype('float') / 255
        labels = np.array(batch['label'])
        yield {
            'image': images,
            'label': labels
        }

In [12]:
for batch in flow_data(dataset['train']):
    break

In [13]:
print(batch['image'].shape, batch['image'].dtype)

(32, 28, 28, 1) float64


In [14]:
print(batch['label'].shape, batch['label'].dtype)

(32,) int64


In [15]:
# https://github.com/google/jax/blob/main/jax/experimental/jax2tf/tests/flax_models/resnet.py

In [16]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""ResNet from Flax example imagenet, see:
https://github.com/google/flax/tree/main/examples/imagenet
"""

# See issue #620.
# pytype: disable=wrong-arg-count

from functools import partial
from typing import Any, Callable, Sequence, Tuple

from flax import linen as nn
import jax.numpy as jnp

ModuleDef = Any


class ResNetBlock(nn.Module):
  """ResNet block."""
  filters: int
  conv: ModuleDef
  norm: ModuleDef
  act: Callable
  strides: Tuple[int, int] = (1, 1)

  @nn.compact
  def __call__(self, x,):
    residual = x
    y = self.conv(self.filters, (3, 3), self.strides)(x)
    y = self.norm()(y)
    y = self.act(y)
    y = self.conv(self.filters, (3, 3))(y)
    y = self.norm(scale_init=nn.initializers.zeros)(y)

    if residual.shape != y.shape:
      residual = self.conv(self.filters, (1, 1),
                           self.strides, name='conv_proj')(residual)
      residual = self.norm(name='norm_proj')(residual)

    return self.act(residual + y)


class BottleneckResNetBlock(nn.Module):
  """Bottleneck ResNet block."""
  filters: int
  conv: ModuleDef
  norm: ModuleDef
  act: Callable
  strides: Tuple[int, int] = (1, 1)

  @nn.compact
  def __call__(self, x):
    residual = x
    y = self.conv(self.filters, (1, 1))(x)
    y = self.norm()(y)
    y = self.act(y)
    y = self.conv(self.filters, (3, 3), self.strides)(y)
    y = self.norm()(y)
    y = self.act(y)
    y = self.conv(self.filters * 4, (1, 1))(y)
    y = self.norm(scale_init=nn.initializers.zeros)(y)

    if residual.shape != y.shape:
      residual = self.conv(self.filters * 4, (1, 1),
                           self.strides, name='conv_proj')(residual)
      residual = self.norm(name='norm_proj')(residual)

    return self.act(residual + y)


class ResNet(nn.Module):
  """ResNetV1."""
  stage_sizes: Sequence[int]
  block_cls: ModuleDef
  num_classes: int
  num_filters: int = 64
  dtype: Any = jnp.float32
  act: Callable = nn.relu
  conv: ModuleDef = nn.Conv

  @nn.compact
  def __call__(self, x, train: bool = True):
    conv = partial(self.conv, use_bias=False, dtype=self.dtype)
    norm = partial(nn.BatchNorm,
                   use_running_average=not train,
                   momentum=0.9,
                   epsilon=1e-5,
                   dtype=self.dtype)

    x = conv(self.num_filters, (7, 7), (2, 2),
             padding=[(3, 3), (3, 3)],
             name='conv_init')(x)
    x = norm(name='bn_init')(x)
    x = nn.relu(x)
    x = nn.max_pool(x, (3, 3), strides=(2, 2), padding='SAME')
    for i, block_size in enumerate(self.stage_sizes):
      for j in range(block_size):
        strides = (2, 2) if i > 0 and j == 0 else (1, 1)
        x = self.block_cls(self.num_filters * 2 ** i,
                           strides=strides,
                           conv=conv,
                           norm=norm,
                           act=self.act)(x)
    x = jnp.mean(x, axis=(1, 2))
    x = nn.Dense(self.num_classes, dtype=self.dtype)(x)
    x = jnp.asarray(x, self.dtype)
    return x


ResNet18 = partial(ResNet, stage_sizes=[2, 2, 2, 2],
                   block_cls=ResNetBlock)
ResNet34 = partial(ResNet, stage_sizes=[3, 4, 6, 3],
                   block_cls=ResNetBlock)
ResNet50 = partial(ResNet, stage_sizes=[3, 4, 6, 3],
                   block_cls=BottleneckResNetBlock)
ResNet101 = partial(ResNet, stage_sizes=[3, 4, 23, 3],
                    block_cls=BottleneckResNetBlock)
ResNet152 = partial(ResNet, stage_sizes=[3, 8, 36, 3],
                    block_cls=BottleneckResNetBlock)
ResNet200 = partial(ResNet, stage_sizes=[3, 24, 36, 3],
                    block_cls=BottleneckResNetBlock)

In [64]:
ResNet18_c10 = partial(ResNet18, num_classes=10)

In [17]:
def cross_entropy_loss(*, logits, labels):
    labels_onehot = jax.nn.one_hot(labels, num_classes=10)
    return optax.softmax_cross_entropy(logits=logits, labels=labels_onehot).mean()

In [18]:
def compute_metrics(*, logits, labels):
    loss = cross_entropy_loss(logits=logits, labels=labels)
    accuracy = jnp.mean(jnp.argmax(logits, -1) == labels)
    metrics = {
        'loss': loss,
        'accuracy': accuracy,
    }
    return metrics

In [65]:
def create_train_state(rng, learning_rate, momentum):
  """Creates initial `TrainState`."""
  net = ResNet18_c10()
  params = net.init(rng, jnp.ones([1, 28, 28, 1]))['params']
  tx = optax.sgd(learning_rate, momentum)
  return train_state.TrainState.create(
      apply_fn=net.apply, params=params, tx=tx)

In [66]:
@jax.jit
def eval_step(params, batch):
    logits, _ = ResNet18_c10().apply({'params': params}, batch['image'], mutable=['batch_stats'])
    return compute_metrics(logits=logits, labels=batch['label'])

In [49]:
def eval_model(params):
    all_metrics = []

    batch_size = 32
    test_dataset = dataset['test']
    total_batch = len(test_dataset) // batch_size

    with tqdm(flow_data(test_dataset, batch_size=batch_size), total=total_batch) as pbar:
        for batch in pbar:
            metrics = eval_step(params, batch)
            metrics = jax.device_get(metrics)
            all_metrics.append(metrics)
    return (
        np.mean([x['loss'].item() for x in all_metrics]),
        np.mean([x['accuracy'].item() for x in all_metrics])
    )

In [67]:
@jax.jit
def train_step(state, batch):
    """Train for a single step."""
    def loss_fn(params):
        logits, _ = ResNet18_c10().apply({'params': params}, batch['image'], mutable=['batch_stats'])
        loss = cross_entropy_loss(logits=logits, labels=batch['label'])
        return loss, logits

    grad_fn = jax.value_and_grad(loss_fn, has_aux=True)
    (_, logits), grads = grad_fn(state.params)
    state = state.apply_gradients(grads=grads)
    metrics = compute_metrics(logits=logits, labels=batch['label'])
    return state, metrics

In [68]:
def train_epoch(state, epoch, rng):
    """Train for a single epoch."""

    batch_metrics = []

    batch_size = 32
    train_dataset = dataset['train']
    total_batch = len(train_dataset) // batch_size

    with tqdm(flow_data(train_dataset, batch_size=batch_size), total=total_batch) as pbar:
        for batch in pbar:
            # breakpoint()
            state, metrics = train_step(state, batch)
            batch_metrics.append(metrics)

            # compute mean of metrics across each batch in epoch.
            batch_metrics_np = jax.device_get(batch_metrics)
            epoch_metrics_np = {
                k: np.mean([metrics[k] for metrics in batch_metrics_np])
                for k in batch_metrics_np[0]}

            pbar.set_description('train epoch: %d, loss: %.4f, accuracy: %.2f' % (
                epoch, epoch_metrics_np['loss'], epoch_metrics_np['accuracy'] * 100))

    return state

In [74]:
rng = jax.random.PRNGKey(0)
rng, init_rng = jax.random.split(rng)

In [ ]:
# weight_after_train <- model, init_weight, data
# predict_label <- model(weight_after_train, predict_to_data)

In [75]:
learning_rate = 0.1
momentum = 0.9
state = create_train_state(init_rng, learning_rate, momentum)

In [76]:
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    # Use a separate PRNG key to permute image data during shuffling
    rng, input_rng = jax.random.split(rng)
    # Run an optimization step over a training batch
    state = train_epoch(state, epoch, input_rng)
    # Evaluate on the test set after each training epoch 
    test_loss, test_accuracy = eval_model(state.params)
    print(' test epoch: %d, loss: %.2f, accuracy: %.2f' % (
        epoch, test_loss, test_accuracy * 100))

  0%|          | 0/1875 [00:00<?, ?it/s]

  0%|          | 0/312 [00:00<?, ?it/s]

 test epoch: 1, loss: 0.08, accuracy: 97.68


  0%|          | 0/1875 [00:00<?, ?it/s]

  0%|          | 0/312 [00:00<?, ?it/s]

...

  0%|          | 0/312 [00:00<?, ?it/s]

 test epoch: 9, loss: 0.06, accuracy: 98.78


  0%|          | 0/1875 [00:00<?, ?it/s]

  0%|          | 0/312 [00:00<?, ?it/s]

 test epoch: 10, loss: 0.05, accuracy: 98.99


In [76]:
# weight_after_train <- model, init_weight, data
# predict_label <- model(weight_after_train, predict_to_data)

In [77]:
for test_batch in flow_data(dataset['test']):
    break

In [78]:
logits, _ = ResNet18_c10().apply({'params': state.params}, test_batch['image'], mutable=['batch_stats'])

In [79]:
jax.device_get(jnp.argmax(logits, -1))

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1], dtype=int32)

In [80]:
test_batch['label']

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4, 9, 6,
       6, 5, 4, 0, 7, 4, 0, 1, 3, 1])